In [1]:
import pandas as pd
import numpy as np
import igraph as ig

# data import

In [2]:
path1 = r'E:\netData\APS\0 citation and paper data\aps-dataset-citations-2010.zip'
f1 = pd.read_csv(path1)

In [3]:
path2 = r'E:\netData\APS\1 Paper\nodepro2010.zip'
f2 = pd.read_csv(path2)

In [4]:
f1 = f1[['citing_doi', 'cited_doi']]

In [5]:
f1['citing_id'] = f1['citing_doi'].map(dict(zip(f2['id'],f2.index)))
f1['cited_id'] =  f1['cited_doi'].map(dict(zip(f2['id'],f2.index)))
f11 = f1[['cited_id','citing_id']].copy()
f11 = f11[f11.columns[::-1]]

In [6]:
g = ig.Graph.DataFrame(f11, directed=True)

In [7]:
g.vcount()

463348

In [8]:
g.ecount()

4745622

In [9]:
g.reciprocity()

0.001745678056000779

# calculate D and CD

In [10]:
def calculate_CD(x):
    if g.vs[x].indegree()== 0:
        return 0,0
    elif g.vs[x].outdegree()== 0:
        return 1,1
    else:
        array = np.array([])
        for successor in g.vs[x].successors():
            array = np.append(array,successor.predecessors())
        array = np.unique(array)
        
        nj = np.intersect1d(array,g.vs[x].predecessors())
        ni = np.setdiff1d(g.vs[x].predecessors(),nj)
        nk = np.setdiff1d(array,g.vs[x].predecessors())

        CD = (len(ni)-len(nj))/(len(ni) + len(nj)+ len(nk))
        
        D = (len(ni))/(len(ni) + len(nj)+ len(nk))
        return [D, CD]

# 2stepD

In [12]:
def calculate_2D(x):
    if g.vs[x].outdegree() == 0:
        return 1, 1

    successors_1 = g.vs[x].successors()
    successors_2 = []

    # Collect successors of successors (second generation)
    for successor_1 in successors_1:
        successors_2.extend(successor_1.successors())

    if len(successors_2) == 0:
        return 1, 1

    array = np.array([])

    # Collect predecessors of successors (second generation)
    for successor_2 in successors_2:
        array = np.append(array, successor_2.predecessors())

    array = np.unique(array)

    nj = np.intersect1d(array, g.vs[x].predecessors())
    ni = np.setdiff1d(g.vs[x].predecessors(), nj)
    nk = np.setdiff1d(array, g.vs[x].predecessors())

    D2 = len(ni)/(len(ni) + len(nj)+ len(nk))
    return D2

# save

In [39]:
M = g.vcount()

In [40]:
Dlst = []
D2lst = []
CDlst = []

In [41]:
for i in range(M):
    DCD = calculate_CD(i)
    Dlst.append(DCD[0])
    CDlst.append(DCD[1])
    D2lst.append(calculate_2D(i))

In [42]:
out_data = [Dlst,D2lst ,CDlst]
name_data = ['D','2stepD','CD']
Ddata = pd.DataFrame(dict(zip(name_data , out_data)))

In [43]:
Ddata.describe()

,D,2stepD,CD
count,463348.000000,463348.000000,463348.000000
mean,0.048016,0.032958,0.023165
std,0.172696,0.162038,0.178672
min,0.000000,0.000000,-1.000000
25%,0.000000,0.000000,-0.013228
50%,0.001054,0.000000,-0.002196
75%,0.012784,0.004800,0.000000
max,1.000000,1.000000,1.000000


In [3]:
f2.describe()

,year,closeness,betweenness,authority,hub,eigenvector,indegree,h,coreness,pagerank,DI,CD,2stepD,D,degree
count,463348.000000,450761.000000,4.633480e+05,4.633480e+05,4.633480e+05,463348.000000,463348.000000,463348.000000,463348.000000,4.633480e+05,374384.000000,463348.000000,463348.000000,463348.000000,463348.000000
mean,1989.869744,0.196733,7.508437e+05,7.363174e-05,7.119738e-03,0.000244,10.242025,3.328967,10.693731,2.158205e-06,-0.004513,0.023165,0.032958,0.048016,20.484051
std,17.557689,0.037913,1.328470e+07,2.548280e-03,4.523288e-02,0.007292,29.829037,4.142201,5.259505,1.142383e-05,0.083491,0.178672,0.162038,0.172696,32.529572
min,1893.000000,0.072611,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,4.297752e-07,-1.000000,-1.000000,0.000000,0.000000,0.000000
25%,1982.000000,0.184324,1.584435e+03,1.566782e-09,6.653263e-07,0.000000,1.000000,1.000000,7.000000,4.736622e-07,-0.017986,-0.013228,0.000000,0.000000,8.000000
50%,1995.000000,0.196080,5.989411e+04,1.333347e-07,1.833919e-05,0.000000,4.000000,2.000000,11.000000,6.484363e-07,-0.004914,-0.002196,0.000000,0.001054,15.000000
75%,2003.000000,0.207086,3.226369e+05,2.036991e-06,2.864381e-04,0.000000,11.000000,5.000000,15.000000,1.244750e-06,0.000000,0.000000,0.004800,0.012784,25.000000
max,2009.000000,1.000000,4.188175e+09,1.000000e+00,1.000000e+00,1.000000,5464.000000,106.000000,34.000000,2.556489e-03,0.995918,1.000000,1.000000,1.000000,5474.000000


In [45]:
f2['CD'] = CDlst
f2['2stepD'] = D2lst
f2['D'] = Dlst

In [49]:
f2['degree'] = g.degree()

In [50]:
f2.to_csv(r'E:\netData\APS\3 properties\nodepro2010.zip',index=False)